In [1]:
#Kindly Note : if import Fails PIP install corresponding library
import json
import requests
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
#Selecting BERT model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
bert_model = SentenceTransformer(model_name)

In [2]:
def FindAlternateGroups_v3(url):

    def GetTitlesLinks(url_link):
        #Set the product collection common URL string
        text1 = 'collections/all/products.json?page='
            #Initialize lists of Titles and Links to products for final JSON array
        all_titles = []
        all_links = []
            #We assume there are MAX 50 pages of products to iterate over each page, this value can be changed
        for i in range(1,50):
                #Strng for Final URL to each product
            final_url = url_link + text1 + str(i)
            print(final_url)
                #Request JSON data
            response = requests.get(final_url)
                #Load JSON into variable
            response_json = json.loads(response.text)
                #Select PRODUCTS from JSON for further analysis
            products_json = response_json['products']
                #If PRODUCTS list is not empty
            if products_json:
                    #Iterate products to get their title and links
                for pattern in products_json:
                    title = pattern['title']
                    links = pattern['handle']
                    all_titles.append(title)
                    all_links.append(links)
            else:
                break
        #print(all_titles)
        #print(all_links)
        return all_titles, all_links
    
    def encodings(titles):
        #Function to get BERT encoding of the specific PRODUCT title
        question = titles
        question_embedded = bert_model.encode(question)
        return question_embedded
    
    def retrieveSimilarProducts(question, question_embedded,sentence_embeddings, sentences, max_sim):
        #Receives the title, title Embedding vectors, original title string and max_sim to filterout Alternate/Similar products
        #Dict for storing individual product alternates
        dict1 = {}
        var = question
        var_2 = url_link + "/products/"
        #List to store links to Laternate products
        li = []
        #Interate over each title and get similarity index, store if has it exceeds max_sim
        for index,pattern_embedding in enumerate(sentence_embeddings):
            sim=cosine_similarity(pattern_embedding.reshape(1, -1),question_embedded.reshape(1, -1))[0][0];
            if ((sim>max_sim) and (sim!=1)):
                #print(index, sim, sentences[index])
                li.append(var_2+links[index])
        dict1.update({var:li})        
        #print(li)
        #print(dict1)
        return dict1

    
    url_link = url
    titles, links = GetTitlesLinks(url_link)
    titles_vectors = bert_model.encode(titles)
    #List to hold the final JSON for output
    dict_main=[]
    #Final iteration, clubbing everything
    for i in range(len(titles)):
        a = encodings(titles[i])
        c = retrieveSimilarProducts(titles[i],a, titles_vectors, titles, max_sim=0.82)
        dict_main.append(c)
    print (dict_main)
    #output to a JSON file
    with open("output.json", "w") as outfile:
        json.dump(dict_main, outfile)


In [3]:
"""Execute this cell last. Change the string of text to the desired URL and code will do the need for you.
"""
text = "https://www.woolsboutiqueuomo.com/"
FindAlternateGroups_v3(url=text)

https://www.woolsboutiqueuomo.com/collections/all/products.json?page=1
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=2
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=3
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=4
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=5
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=6
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=7
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=8
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=9
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=10
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=11
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=12
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=13
https://www.woolsboutiqueuomo.com/collections/all/products.json?page=14
h